In [2]:
import pandas as pd
from pathlib import Path

In [4]:
month = 1
dataset_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-{month:02}.csv.gz"
df_chunks = pd.read_csv(dataset_url,compression='gzip')
df_chunks.shape()

URLError: <urlopen error [Errno 110] Connection timed out>

In [ ]:
df_chunks.shape()

NameError: name 'df_chunks' is not defined

In [3]:
df = next(df_chunks)
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00001,2019-01-01 00:30:00,2019-01-01 02:51:55,NaN,NaN,NaN,B00001
1,B00001,2019-01-01 00:45:00,2019-01-01 00:54:49,NaN,NaN,NaN,B00001
2,B00001,2019-01-01 00:15:00,2019-01-01 00:54:52,NaN,NaN,NaN,B00001
3,B00008,2019-01-01 00:19:00,2019-01-01 00:39:00,NaN,NaN,NaN,B00008
4,B00008,2019-01-01 00:27:00,2019-01-01 00:37:00,NaN,NaN,NaN,B00008
...,...,...,...,...,...,...,...
99995,B02869,2019-01-01 01:46:01,2019-01-01 01:54:10,29.0,149.0,NaN,B02869
99996,B02869,2019-01-01 01:57:38,2019-01-01 02:02:52,210.0,150.0,NaN,B02869
99997,B02869,2019-01-01 01:10:03,2019-01-01 01:31:16,38.0,73.0,NaN,B02869
99998,B02869,2019-01-01 01:36:45,2019-01-01 01:52:01,9.0,223.0,NaN,B02869


In [4]:
output_path = 'data/local.parquet'
df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
df.to_parquet(output_path, index=False)

In [5]:
for i, chunk in enumerate(df_chunks, start=1):
    existing = pd.read_parquet(output_path)
    chunk.pickup_datetime = pd.to_datetime(chunk.pickup_datetime)
    chunk.dropOff_datetime = pd.to_datetime(chunk.dropOff_datetime)
    combined = pd.concat([existing, chunk], ignore_index=True)
    combined.to_parquet(output_path, index=False)
    print(f'Inserted chunk {i}')
print('Chunks inserted successfully')

Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5
Inserted chunk 6
Inserted chunk 7
Inserted chunk 8
Inserted chunk 9
Inserted chunk 10
Inserted chunk 11
Inserted chunk 12
Inserted chunk 13
Inserted chunk 14
Inserted chunk 15
Inserted chunk 16
Inserted chunk 17
Inserted chunk 18
Inserted chunk 19
Inserted chunk 20
Inserted chunk 21
Inserted chunk 22
Inserted chunk 23
Inserted chunk 24
Inserted chunk 25
Inserted chunk 26
Inserted chunk 27
Inserted chunk 28
Inserted chunk 29
Inserted chunk 30
Inserted chunk 31
Inserted chunk 32
Inserted chunk 33
Inserted chunk 34
Inserted chunk 35
Inserted chunk 36
Inserted chunk 37
Inserted chunk 38
Inserted chunk 39
Inserted chunk 40
Inserted chunk 41
Inserted chunk 42
Inserted chunk 43


KeyboardInterrupt: 

In [7]:
while True:
    try:
       df = next(df_chunks)
       df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
       df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
       df.to_parquet(output_path, mode='append', index=False)
       print('Inserted another chunk')
    except StopIteration:
        print('Finished inserting data. Thank you')
        break

TypeError: __cinit__() got an unexpected keyword argument 'mode'

In [3]:
df = pd.read_csv("fhv_tripdata_2019-01.csv", nrows=100)

In [4]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00001,2019-01-01 00:30:00,2019-01-01 02:51:55,NaN,NaN,NaN,B00001
1,B00001,2019-01-01 00:45:00,2019-01-01 00:54:49,NaN,NaN,NaN,B00001
2,B00001,2019-01-01 00:15:00,2019-01-01 00:54:52,NaN,NaN,NaN,B00001
3,B00008,2019-01-01 00:19:00,2019-01-01 00:39:00,NaN,NaN,NaN,B00008
4,B00008,2019-01-01 00:27:00,2019-01-01 00:37:00,NaN,NaN,NaN,B00008
...,...,...,...,...,...,...,...
95,B00248,2019-01-01 00:11:09,2019-01-01 00:20:59,NaN,265.0,NaN,B00248
96,B00248,2019-01-01 00:52:34,2019-01-01 01:01:06,NaN,265.0,NaN,B00248
97,B00248,2019-01-01 00:17:44,2019-01-01 00:24:03,NaN,265.0,NaN,B00248
98,B00248,2019-01-01 00:00:10,2019-01-01 00:33:26,NaN,265.0,NaN,B00248


In [5]:
print(pd.io.sql.get_schema(df, name="fhv_trips_01"))

CREATE TABLE "fhv_trips_01" (
"dispatching_base_num" TEXT,
  "pickup_datetime" TEXT,
  "dropOff_datetime" TEXT,
  "PUlocationID" REAL,
  "DOlocationID" REAL,
  "SR_Flag" REAL,
  "Affiliated_base_number" TEXT
)


In [6]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)

In [7]:
print(pd.io.sql.get_schema(df, name="fhv_trips_01"))

CREATE TABLE "fhv_trips_01" (
"dispatching_base_num" TEXT,
  "pickup_datetime" TIMESTAMP,
  "dropOff_datetime" TIMESTAMP,
  "PUlocationID" REAL,
  "DOlocationID" REAL,
  "SR_Flag" REAL,
  "Affiliated_base_number" TEXT
)
